In [ ]:
import numpy as np
from scipy.io import wavfile
from matplotlib import pyplot as plt

In [ ]:
def mix_audio(signal, noise, snr):
    '''
    Adapted from https://stackoverflow.com/a/72124325
    '''
    # if the audio is longer than the noise
    # play the noise in repeat for the duration of the audio
    noise = noise[np.arange(len(signal)) % len(noise)]
    
    # if the audio is shorter than the noi
    # this is important if loading resulted in 
    # uint8 or uint16 types, because it would cause overflow
    # when squaring and calculating mean
    noise = noise.astype(np.float32)
    signal = signal.astype(np.float32)
    
    # get the initial energy for reference
    signal_energy = np.mean(signal**2)
    noise_energy = np.mean(noise**2)
    # calculates the gain to be applied to the noise 
    # to achieve the given SNR
    g = np.sqrt(10.0 ** (-snr/10) * signal_energy / noise_energy)
    
    # Assumes signal and noise to be decorrelated
    # and calculate (a, b) such that energy of 
    # a*signal + b*noise matches the energy of the input signal
    a = np.sqrt(1 / (1 + g**2))
    b = np.sqrt(g**2 / (1 + g**2))

    c = a * signal + b * noise

    print(g, a, b)
    # mix the signals
    return np.int16(c/np.max(np.abs(c)) * 32767)



In [ ]:
# signal = np.random.randint(0, 2, 10**7) - 0.5
sampling_rate, signal = wavfile.read("C:\\Users\\23963\\Desktop\\NATO\\NATO_Phonetic_Alphabet_reading_mono.wav")
# use some non-standard noise distribution
# noise = np.sin(np.random.randn(6*10**7))
noise = np.random.normal(0, 1, size=len(signal))

# noise = np.ones_like(signal)*.000001

for n in range(-6,7):
    noisy = mix_audio(signal, noise, 2**n)
    # plt.hist(noisy, bins=300)

    wavfile.write("C:\\Users\\23963\\Desktop\\NATO\\NATO_Phonetic_Alphabet_reading_mono_SNR_{}.wav".format(round(2**n,3)),sampling_rate, noisy)

